In [4]:
from dotenv import load_dotenv
import os

# .env load 
load_dotenv()

# check API KEY 
if "OPENAI_API_KEY" not in os.environ:
    print("API KEY 정보가 없습니다. 확인 후 환경변수에 등록해주세요.")

# PDF 불러오기

In [8]:
# pip install pypdf
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./test.pdf")
print(loader)
document = loader.load()
print(document[0].page_content[:200])

제 안 요 청 서        
사 업 명 생성형 AI법령정보서비스 정보화전략계획 수립
발주기관 법 제 처
2024. 4.
사 업
담당자기획조정관실
법령데이터
혁신팀성 명 전화번호 e-mail
과  장이 영 진 044-200-6782 aldjlsd@korea.kr
사무관 하 선 우 044-200-6784 pasero@korea.krⅠ. 사업 개요
1.입찰 


# 텍스트 분할하기

In [12]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",
    chunk_size=3000,
    chunk_overlap=500
)

split_docs = text_splitter.split_documents(document)
print(len(split_docs))

38


# 요약

In [17]:
from langchain.prompts import PromptTemplate 
from langchain_openai import ChatOpenAI 
from langchain.chains import LLMChain

map_template = """
    COTENTS의 주요 내용을 요약해주세요

    CONTENTS: {pages}

    ANSWER: 
"""

map_prompt = PromptTemplate.from_template(map_template)

llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo-16k'
)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# 요약 통합

In [19]:
reduce_template = """
    요약의 집합들입니다. 집합들을 통합하여 요약해주세요.

    DOCS: {doc_summaries}

    ANSWER: 
"""

reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [20]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain 
from langchain.chains import ReduceDocumentsChain 

combine_documents_chain = StuffDocumentsChain(
    llm_chain = reduce_chain,
    document_variable_name= "doc_summaries"
)

reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain = combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max = 4000
)

# 통합 체인

In [21]:
from langchain.chains import MapReduceDocumentsChain 

map_reduce_chain = MapReduceDocumentsChain(
    llm_chain = map_chain, 
    reduce_documents_chain = reduce_documents_chain,
    document_variable_name = "pages",
    return_intermediate_steps = False
)

In [23]:
result = map_reduce_chain.invoke(split_docs)

print(result)

{'input_documents': [Document(page_content='제 안 요 청 서        \n사 업 명 생성형 AI법령정보서비스 정보화전략계획 수립\n발주기관 법 제 처\n2024. 4.\n사 업\n담당자기획조정관실\n법령데이터\n혁신팀성 명 전화번호 e-mail\n과  장이 영 진 044-200-6782 aldjlsd@korea.kr\n사무관 하 선 우 044-200-6784 pasero@korea.krⅠ. 사업 개요\n1.입찰 일반 사항·······························································································1\n2.주요 사업 내용·······························································································1\nⅡ. 추진 현황 및 개선방향\n1.지능형 법령검색 서비스 현황·····································································2\n2.문제점 및 개선방향·······················································································2\n3.추진방향 및 전략···························································································3\nⅢ. 사업 내용···········································································································4\nⅣ. 사업 대상 시스템 현황\n1.국가법령정보센터 ············································································

In [201]:
result['input_documents'] = str(result['input_documents'])

In [204]:
import json
with open("result.json", "w", encoding='utf-8') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)

# PydanticOutputParser

In [405]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

In [435]:
llm = ChatOpenAI(
    temperature = 0,
    model_name = "gpt-3.5-turbo"
)

In [462]:
class EmailSummary(BaseModel):
    person: str = Field(description="Sender name")
    email: str = Field(description="Sender email address")
    subject: str = Field(description="Mail subject")
    summary: str = Field(description="Mail summary")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [463]:
prompt = PromptTemplate.from_template(""" 
# QUESTION 
{question}

# EMAIL TEXT
{email_text}

# FORMAT
{format}
""")

prompt = prompt.partial(format=parser.get_format_instructions())        # 프롬프트에 부분적으로 적용

In [464]:
email_text = """From : "김명섭"<kms4737@nhn.com>
To : narae3759@ppsystem.co.kr
Date : 2024/04/11 목요일 오후 6:17:07
Subject : [AICA] 2024년 1분기 국가 AI 데이터센터 서비스 만족도 설문조사

본 메일은 2024년 국가 AI데이터센터 서비스 이용자에게 보내는 메일입니다.

안녕하세요 국가인공지능데이터센터입니다.

서비스 개선을 위한 서비스 이용자 만족도 조사 실시에 대해 안내드립니다.

서비스 이용자 만족도 조사(~4/15)

사업단에서는 이용자 분들의 의견을 수렴하여 제공 중인 서비스를 개선하기 위해 만족도 조사를 실시하고 있사오니 많은 참여 부탁드립니다.

첨부파일을 작성 후 하기 메일로 회신해주시기 바랍니다.

 □ 참여방법 : 첨부파일 작성 후 본 메일 회신(kms4737@nhn.com)

  - 한글 파일 수정 및 출력 스캔본 가능

 □ 첨부파일 : 2024년 1분기 국가 AI 데이터센터 서비스 만족도 설문조사

 □ 마감기한 : ~4/15(월)


감사합니다.
"""

In [465]:
chain = prompt | llm 

response = chain.invoke(
    {
        "question": "이메일의 내용에서 주요 내용을 추출해주세요",
        "email_text": email_text
    }
)

print(response.content)

{
  "person": "김명섭",
  "email": "kms4737@nhn.com",
  "subject": "[AICA] 2024년 1분기 국가 AI 데이터센터 서비스 만족도 설문조사",
  "summary": "본 메일은 2024년 국가 AI데이터센터 서비스 이용자에게 보내는 메일입니다. 안녕하세요 국가인공지능데이터센터입니다. 서비스 개선을 위한 서비스 이용자 만족도 조사 실시에 대해 안내드립니다. 서비스 이용자 만족도 조사(~4/15). 사업단에서는 이용자 분들의 의견을 수렴하여 제공 중인 서비스를 개선하기 위해 만족도 조사를 실시하고 있사오니 많은 참여 부탁드립니다."
}


In [466]:
chain = prompt | llm | parser 

response = chain.invoke(
    {
        "question": "이메일의 내용에서 주요 내용을 추출해주세요",
        "email_text": email_text
    }
)

response

EmailSummary(person='김명섭', email='kms4737@nhn.com', subject='[AICA] 2024년 1분기 국가 AI 데이터센터 서비스 만족도 설문조사', summary='본 메일은 2024년 국가 AI데이터센터 서비스 이용자에게 보내는 메일입니다. 안내드립니다. 서비스 개선을 위한 서비스 이용자 만족도 조사 실시에 대해 안내드립니다. 서비스 이용자 만족도 조사(~4/15). 사업단에서는 이용자 분들의 의견을 수렴하여 제공 중인 서비스를 개선하기 위해 만족도 조사를 실시하고 있사오니 많은 참여 부탁드립니다.')

# CommaSeparatedListOutputParser

In [467]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

In [474]:
llm = ChatOpenAI(
    temperature = 0,
    model_name = "gpt-3.5-turbo"
)

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template = """CONTENT에 맞는 인스타그램의 태그를 10개 추천해주세요. 
    
    CONTENT: {content}

    {format_instructions}""",
    input_variables = ["content"],
    partial_variables = {"format_instructions": format_instructions}
)

chain = prompt | llm | output_parser

In [475]:
response = chain.invoke(
    {
        "content": "여행중. 역시 여행은 오션뷰!"
    }
)

response

['#여행',
 '#오션뷰',
 '#해변',
 '#바다',
 '#휴가',
 '#여행스타그램',
 '#해수욕',
 '#해변가',
 '#해외여행',
 '#풍경사진']